In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM

# Datasets
##Cat_versus_dog

In [7]:
catdog_builder = tfds.builder("cats_vs_dogs")
catdog_builder.download_and_prepare()

catdog_train_dataset, catdog_test_dataset = catdog_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

catdog_num_classes = catdog_builder.info.features['label'].num_classes
catdog_data_type = 'text' if 'text' in catdog_builder.info.features else 'image'

catdog_train_dataset = catdog_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
catdog_test_dataset = catdog_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(catdog_train_dataset))
print(tf.data.experimental.cardinality(catdog_test_dataset))
print(catdog_num_classes)
print(catdog_data_type)

tf.Tensor(18610, shape=(), dtype=int64)
tf.Tensor(4652, shape=(), dtype=int64)
2
image


##Flowers

In [0]:
flower_builder = tfds.builder("tf_flowers")
flower_builder.download_and_prepare()

flower_train_dataset, flower_test_dataset = flower_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

flower_num_classes = flower_builder.info.features['label'].num_classes
flower_data_type = 'text' if 'text' in flower_builder.info.features else 'image'

flower_train_dataset = flower_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
flower_test_dataset = flower_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(flower_train_dataset))
print(tf.data.experimental.cardinality(flower_test_dataset))
print(flower_num_classes)
print(flower_data_type)

tf.Tensor(2936, shape=(), dtype=int64)
tf.Tensor(734, shape=(), dtype=int64)
5
image


##Malaria

In [0]:
malaria_builder = tfds.builder("malaria")
malaria_builder.download_and_prepare()

malaria_train_dataset, malaria_test_dataset = malaria_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

malaria_num_classes = malaria_builder.info.features['label'].num_classes
malaria_data_type = 'text' if 'text' in malaria_builder.info.features else 'image'

malaria_train_dataset = malaria_train_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))
malaria_test_dataset = malaria_test_dataset.map(lambda img, label: (tf.image.resize(img, [64, 64]) / 255.0, label))

print(tf.data.experimental.cardinality(malaria_train_dataset))
print(tf.data.experimental.cardinality(malaria_test_dataset))
print(malaria_num_classes)
print(malaria_data_type)

tf.Tensor(22046, shape=(), dtype=int64)
tf.Tensor(5512, shape=(), dtype=int64)
2
image


##Imdb

In [5]:
imdb_builder = tfds.builder("imdb_reviews/subwords8k")
imdb_builder.download_and_prepare()

imdb_train_dataset, imdb_test_dataset = imdb_builder.as_dataset(
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

imdb_encoder = imdb_builder.info.features['text'].encoder
imdb_num_classes = imdb_builder.info.features['label'].num_classes
imdb_data_type = 'text' if 'text' in imdb_builder.info.features else 'image'

print(tf.data.experimental.cardinality(imdb_train_dataset))
print(tf.data.experimental.cardinality(imdb_test_dataset))
print(imdb_num_classes)
print(imdb_data_type)

tf.Tensor(20000, shape=(), dtype=int64)
tf.Tensor(5000, shape=(), dtype=int64)
2
text


##Yelp
Large Yelp Review Dataset. This is a dataset for binary sentiment classification.

In [6]:
yelp_builder = tfds.builder("yelp_polarity_reviews/subwords8k")
yelp_builder.download_and_prepare()

yelp_train_dataset, yelp_test_dataset = yelp_builder.as_dataset(
    split=['train[:40%]', 'train[40%:50%]'],
    as_supervised=True
)

yelp_encoder = yelp_builder.info.features['text'].encoder
yelp_num_classes = yelp_builder.info.features['label'].num_classes
yelp_data_type = 'text' if 'text' in yelp_builder.info.features else 'image'

print(tf.data.experimental.cardinality(yelp_train_dataset))
print(tf.data.experimental.cardinality(yelp_test_dataset))
print(yelp_num_classes)
print(yelp_data_type)

tf.Tensor(224000, shape=(), dtype=int64)
tf.Tensor(56000, shape=(), dtype=int64)
2
text


# Model
define a CNN model.

In [0]:
def cnn_model(num_classes):
  model = tf.keras.Sequential([
      Conv2D(32, (3, 3), activation='relu',
             kernel_regularizer=tf.keras.regularizers.l2(0.001),
             input_shape=(64, 64, 3)),
      MaxPooling2D((2, 2)),
      Dropout(0.2),
      Conv2D(64, (3, 3), activation='relu',
             kernel_regularizer=tf.keras.regularizers.l2(0.001)),
      MaxPooling2D((2, 2)),
      Dropout(0.2),
      Conv2D(128, (3, 3), activation='relu',
             kernel_regularizer=tf.keras.regularizers.l2(0.001)),
      MaxPooling2D((2, 2)),
      Dropout(0.2),
      Flatten(),
      Dense(128, activation='relu'),
      Dense(64, activation='relu'),
      Dense(num_classes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

define a RNN model.

In [0]:
def rnn_model(encoder, num_classes):
  model = tf.keras.Sequential([
      Embedding(encoder.vocab_size, 64),
      Bidirectional(LSTM(64, return_sequences=True)),
      Bidirectional(LSTM(32)),
      Dense(64, activation='relu'),
      Dropout(0.5),
      Dense(1)
  ])

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])
  
  return model

# Shuffling

In [0]:
def shuffle_test(train_dataset, test_dataset, batch_size = 50, epochs = 10, num_classes = 2, data_type = 'image', encoder = None):
  train_dataset_size = int(tf.data.experimental.cardinality(train_dataset))
  test_dataset_size = int(tf.data.experimental.cardinality(test_dataset))

  if data_type == 'image':
    shuffle_test_dataset = test_dataset.batch(batch_size).repeat()
    t1_shuffle = time.time()
    shuffle_train_dataset = train_dataset.shuffle(1000).batch(batch_size).repeat()
    t2_shuffle = time.time()

    model_shuffle = cnn_model(num_classes)
  
  if data_type == 'text':
    shuffle_test_dataset = test_dataset.padded_batch(batch_size).repeat()
    t1_shuffle = time.time()
    shuffle_train_dataset = train_dataset.shuffle(1000).padded_batch(batch_size).repeat()
    t2_shuffle = time.time()

    model_shuffle = rnn_model(encoder, num_classes)
  
  print("shuffling takes", t2_shuffle-t1_shuffle, "seconds")

  t3_shuffle = time.time()
  model_shuffle.fit(
    shuffle_train_dataset,
    steps_per_epoch = train_dataset_size // batch_size,
    validation_data = shuffle_test_dataset,
    validation_steps = test_dataset_size // batch_size,
    epochs = epochs
  )
  t4_shuffle = time.time()
  print("training based on shuffling takes", t4_shuffle-t3_shuffle, "seconds")

##Cat_versus_dog

In [0]:
shuffle_test(catdog_train_dataset, catdog_test_dataset, num_classes=catdog_num_classes, data_type = catdog_data_type)

shuffling takes 0.0012943744659423828 seconds
Epoch 1/10
372/372 [==============================] - 17s 45ms/step - loss: 0.6996 - accuracy: 0.5928 - val_loss: 0.6336 - val_accuracy: 0.6488
Epoch 2/10
372/372 [==============================] - 16s 44ms/step - loss: 0.5878 - accuracy: 0.7048 - val_loss: 0.5493 - val_accuracy: 0.7357
Epoch 3/10
372/372 [==============================] - 16s 44ms/step - loss: 0.5355 - accuracy: 0.7429 - val_loss: 0.5030 - val_accuracy: 0.7712
Epoch 4/10
372/372 [==============================] - 17s 45ms/step - loss: 0.5034 - accuracy: 0.7691 - val_loss: 0.4748 - val_accuracy: 0.7942
Epoch 5/10
372/372 [==============================] - 16s 42ms/step - loss: 0.4834 - accuracy: 0.7840 - val_loss: 0.4464 - val_accuracy: 0.8170
Epoch 6/10
372/372 [==============================] - 15s 41ms/step - loss: 0.4613 - accuracy: 0.8003 - val_loss: 0.4468 - val_accuracy: 0.8153
Epoch 7/10
372/372 [==============================] - 18s 49ms/step - loss: 0.4469 - accur

##Flowers

In [0]:
shuffle_test(flower_train_dataset, flower_test_dataset, num_classes=flower_num_classes, data_type = flower_data_type)

shuffling takes 0.0015208721160888672 seconds
Epoch 1/10
58/58 [==============================] - 2s 39ms/step - loss: 1.5632 - accuracy: 0.3421 - val_loss: 1.3128 - val_accuracy: 0.4800
Epoch 2/10
58/58 [==============================] - 2s 41ms/step - loss: 1.2454 - accuracy: 0.5003 - val_loss: 1.2206 - val_accuracy: 0.5243
Epoch 3/10
58/58 [==============================] - 2s 41ms/step - loss: 1.1494 - accuracy: 0.5655 - val_loss: 1.1290 - val_accuracy: 0.5729
Epoch 4/10
58/58 [==============================] - 2s 42ms/step - loss: 1.0551 - accuracy: 0.5863 - val_loss: 1.0066 - val_accuracy: 0.6371
Epoch 5/10
58/58 [==============================] - 2s 41ms/step - loss: 0.9985 - accuracy: 0.6317 - val_loss: 1.0291 - val_accuracy: 0.6043
Epoch 6/10
58/58 [==============================] - 2s 41ms/step - loss: 0.9574 - accuracy: 0.6424 - val_loss: 0.9261 - val_accuracy: 0.6400
Epoch 7/10
58/58 [==============================] - 2s 41ms/step - loss: 0.9046 - accuracy: 0.6674 - val_los

##Malaria

In [0]:
shuffle_test(malaria_train_dataset, malaria_test_dataset, num_classes = malaria_num_classes, data_type = malaria_data_type)

shuffling takes 0.0013654232025146484 seconds
Epoch 1/10
440/440 [==============================] - 15s 35ms/step - loss: 0.5061 - accuracy: 0.7573 - val_loss: 0.2229 - val_accuracy: 0.9376
Epoch 2/10
440/440 [==============================] - 16s 35ms/step - loss: 0.2089 - accuracy: 0.9400 - val_loss: 0.1766 - val_accuracy: 0.9551
Epoch 3/10
440/440 [==============================] - 16s 36ms/step - loss: 0.1732 - accuracy: 0.9527 - val_loss: 0.1627 - val_accuracy: 0.9587
Epoch 4/10
440/440 [==============================] - 15s 35ms/step - loss: 0.1621 - accuracy: 0.9544 - val_loss: 0.1598 - val_accuracy: 0.9569
Epoch 5/10
440/440 [==============================] - 15s 34ms/step - loss: 0.1531 - accuracy: 0.9572 - val_loss: 0.1569 - val_accuracy: 0.9558
Epoch 6/10
440/440 [==============================] - 16s 37ms/step - loss: 0.1480 - accuracy: 0.9583 - val_loss: 0.1563 - val_accuracy: 0.9565
Epoch 7/10
440/440 [==============================] - 15s 34ms/step - loss: 0.1440 - accur

##Imdb

In [0]:
shuffle_test(
    imdb_train_dataset,
    imdb_test_dataset,
    batch_size = 200,
    num_classes = imdb_num_classes,
    data_type = imdb_data_type,
    encoder = imdb_encoder,
    epochs = 5
)

shuffling takes 0.002113819122314453 seconds
Epoch 1/5
100/100 [==============================] - 86s 856ms/step - loss: 0.6931 - accuracy: 0.5016 - val_loss: 0.6926 - val_accuracy: 0.4938
Epoch 2/5
100/100 [==============================] - 84s 843ms/step - loss: 0.6837 - accuracy: 0.5034 - val_loss: 0.6151 - val_accuracy: 0.5856
Epoch 3/5
100/100 [==============================] - 84s 844ms/step - loss: 0.4677 - accuracy: 0.7757 - val_loss: 0.3724 - val_accuracy: 0.8376
Epoch 4/5
100/100 [==============================] - 85s 845ms/step - loss: 0.3171 - accuracy: 0.8771 - val_loss: 0.3521 - val_accuracy: 0.8468
Epoch 5/5
100/100 [==============================] - 85s 846ms/step - loss: 0.2553 - accuracy: 0.9093 - val_loss: 0.3387 - val_accuracy: 0.8636
training based on shuffling takes 433.5717399120331 seconds


Yelp

In [0]:
shuffle_test(
    yelp_train_dataset,
    yelp_test_dataset,
    batch_size = 500,
    num_classes = yelp_num_classes,
    data_type = yelp_data_type,
    encoder = yelp_encoder,
    epochs = 3
)

shuffling takes 0.0020787715911865234 seconds
Epoch 1/3
448/448 [==============================] - 466s 1s/step - loss: 0.4305 - accuracy: 0.7599 - val_loss: 0.2193 - val_accuracy: 0.9171
Epoch 2/3
448/448 [==============================] - 462s 1s/step - loss: 0.2067 - accuracy: 0.9244 - val_loss: 0.1911 - val_accuracy: 0.9253
Epoch 3/3
448/448 [==============================] - 463s 1s/step - loss: 0.1798 - accuracy: 0.9352 - val_loss: 0.1860 - val_accuracy: 0.9275
training based on shuffling takes 1401.9833824634552 seconds


# SWO

In [0]:
def generate_batches_by_swo(dataset, batch_size, data_type = 'image', test_mode = False):
    dataset_size = int(tf.data.experimental.cardinality(dataset))
    steps = dataset_size // batch_size
    indexes = [np.random.choice(dataset_size, size=batch_size, replace=False) for _ in range(steps)]

    if test_mode:
      returned_indexes = indexes.copy()
    
    batch_group_by_index = {}
    for batch_index in range(steps):
      for element_index in indexes[batch_index]:
        if element_index not in batch_group_by_index:
          batch_group_by_index[element_index] = [batch_index]
        else:
          batch_group_by_index[element_index].append(batch_index)

    element_index = 0
    batch_features = [[] for _ in range(steps)]
    batch_labels = [[] for _ in range(steps)]
    
    if data_type == 'image':
      for element in dataset.as_numpy_iterator():
        if element_index in batch_group_by_index:
          for batch_index in batch_group_by_index[element_index]:
            batch_features[batch_index].append(element[0])
            batch_labels[batch_index].append(element[1])
        element_index += 1
    
      features_shape = [batch_size, 64, 64, 3]
    
    if data_type == 'text':
      max_text_len = 0
      for element in dataset.as_numpy_iterator():
        if element_index in batch_group_by_index:
          for batch_index in batch_group_by_index[element_index]:
            batch_features[batch_index].append(element[0])
            batch_labels[batch_index].append(element[1])
            max_text_len = max(len(element[0]), max_text_len)
        element_index += 1
      
      #padding
      for bf in batch_features:
        for i in range(len(bf)):
          bf[i] = np.pad(
              bf[i],
              pad_width=(0, max_text_len-len(bf[i])),
              mode = 'constant',
              constant_values=0
          )
      
      features_shape = [batch_size, max_text_len]

    print("transforming data to dataset object")

    labels_shape = [batch_size, ]
    
    dataset = tf.data.Dataset.from_generator(
        lambda: zip(batch_features, batch_labels),
        (tf.float32, tf.float32),
        (features_shape, labels_shape)
    )

    if not test_mode:
      return dataset
    else:
      return dataset, returned_indexes

In [0]:
def swo_test(train_dataset, test_dataset, batch_size = 50, epochs = 10, data_type = 'image', num_classes = 2, encoder = None):
  train_dataset_size = int(tf.data.experimental.cardinality(train_dataset))
  test_dataset_size = int(tf.data.experimental.cardinality(test_dataset))
  train_steps = train_dataset_size // batch_size
  test_steps = test_dataset_size // batch_size

  if data_type == 'image':
    swo_test_dataset = test_dataset.batch(batch_size).repeat()
    model_swo = cnn_model(num_classes)
  
  if data_type == 'text':
    swo_test_dataset = test_dataset.padded_batch(batch_size).repeat()
    model_swo = rnn_model(encoder, num_classes)

  t1_swo = time.time()
  swo_train_dataset = generate_batches_by_swo(train_dataset, batch_size=batch_size, data_type=data_type).repeat()
  t2_swo = time.time()
  print("swo takes", t2_swo-t1_swo, "seconds")

  t3_swo = time.time()
  model_swo.fit(
    swo_train_dataset,
    steps_per_epoch = train_steps,
    validation_data = swo_test_dataset,
    validation_steps = test_steps,
    epochs=epochs
  )
  t4_swo = time.time()
  print("training based on swo takes", t4_swo-t3_swo, "seconds")

##Cat_versus_dog

In [0]:
swo_test(catdog_train_dataset, catdog_test_dataset, num_classes = catdog_num_classes, data_type = catdog_data_type)

transforming data to dataset object
swo takes 10.514053583145142 seconds
Epoch 1/10
372/372 [==============================] - 5s 13ms/step - loss: 0.6991 - accuracy: 0.6022 - val_loss: 0.6051 - val_accuracy: 0.6903
Epoch 2/10
372/372 [==============================] - 5s 13ms/step - loss: 0.5897 - accuracy: 0.7084 - val_loss: 0.5384 - val_accuracy: 0.7533
Epoch 3/10
372/372 [==============================] - 5s 13ms/step - loss: 0.5342 - accuracy: 0.7532 - val_loss: 0.5315 - val_accuracy: 0.7583
Epoch 4/10
372/372 [==============================] - 5s 13ms/step - loss: 0.5011 - accuracy: 0.7780 - val_loss: 0.4972 - val_accuracy: 0.7888
Epoch 5/10
372/372 [==============================] - 5s 13ms/step - loss: 0.4693 - accuracy: 0.8007 - val_loss: 0.4691 - val_accuracy: 0.7998
Epoch 6/10
372/372 [==============================] - 5s 13ms/step - loss: 0.4487 - accuracy: 0.8137 - val_loss: 0.5147 - val_accuracy: 0.7701
Epoch 7/10
372/372 [==============================] - 5s 13ms/step - 

##Flowers

In [0]:
swo_test(flower_train_dataset, flower_test_dataset, num_classes = flower_num_classes, data_type = flower_data_type)

transforming data to dataset object
swo takes 2.175234317779541 seconds
Epoch 1/10
58/58 [==============================] - 1s 19ms/step - loss: 1.5302 - accuracy: 0.3579 - val_loss: 1.2820 - val_accuracy: 0.5129
Epoch 2/10
58/58 [==============================] - 1s 17ms/step - loss: 1.2079 - accuracy: 0.5314 - val_loss: 1.1861 - val_accuracy: 0.5814
Epoch 3/10
58/58 [==============================] - 1s 17ms/step - loss: 1.0877 - accuracy: 0.6014 - val_loss: 1.1205 - val_accuracy: 0.5771
Epoch 4/10
58/58 [==============================] - 1s 17ms/step - loss: 0.9925 - accuracy: 0.6459 - val_loss: 1.0955 - val_accuracy: 0.5886
Epoch 5/10
58/58 [==============================] - 1s 17ms/step - loss: 0.9194 - accuracy: 0.6610 - val_loss: 1.0018 - val_accuracy: 0.6343
Epoch 6/10
58/58 [==============================] - 1s 17ms/step - loss: 0.8311 - accuracy: 0.7028 - val_loss: 0.9779 - val_accuracy: 0.6543
Epoch 7/10
58/58 [==============================] - 1s 17ms/step - loss: 0.7652 - 

##Malaria

In [0]:
swo_test(malaria_train_dataset, malaria_test_dataset, num_classes = malaria_num_classes, data_type = malaria_data_type)

transforming data to dataset object
swo takes 14.002123832702637 seconds
Epoch 1/10
440/440 [==============================] - 10s 22ms/step - loss: 0.4643 - accuracy: 0.7895 - val_loss: 0.2518 - val_accuracy: 0.9269
Epoch 2/10
440/440 [==============================] - 10s 22ms/step - loss: 0.2023 - accuracy: 0.9427 - val_loss: 0.1834 - val_accuracy: 0.9540
Epoch 3/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1690 - accuracy: 0.9536 - val_loss: 0.1753 - val_accuracy: 0.9556
Epoch 4/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1548 - accuracy: 0.9566 - val_loss: 0.1771 - val_accuracy: 0.9545
Epoch 5/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1481 - accuracy: 0.9585 - val_loss: 0.1633 - val_accuracy: 0.9545
Epoch 6/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1395 - accuracy: 0.9595 - val_loss: 0.1633 - val_accuracy: 0.9564
Epoch 7/10
440/440 [==============================] - 10s 22ms/

##Imdb

In [0]:
swo_test(
    imdb_train_dataset,
    imdb_test_dataset,
    batch_size = 200,
    num_classes = imdb_num_classes,
    data_type = imdb_data_type,
    encoder = imdb_encoder,
    epochs = 5
)

transforming data to dataset object
swo takes 7.309563159942627 seconds
Epoch 1/5
100/100 [==============================] - 182s 2s/step - loss: 0.6929 - accuracy: 0.5011 - val_loss: 0.6922 - val_accuracy: 0.4938
Epoch 2/5
100/100 [==============================] - 181s 2s/step - loss: 0.6496 - accuracy: 0.5602 - val_loss: 0.5022 - val_accuracy: 0.7604
Epoch 3/5
100/100 [==============================] - 181s 2s/step - loss: 0.3705 - accuracy: 0.8439 - val_loss: 0.3761 - val_accuracy: 0.8508
Epoch 4/5
100/100 [==============================] - 180s 2s/step - loss: 0.2337 - accuracy: 0.9191 - val_loss: 0.4159 - val_accuracy: 0.8546
Epoch 5/5
100/100 [==============================] - 180s 2s/step - loss: 0.1788 - accuracy: 0.9441 - val_loss: 0.4119 - val_accuracy: 0.8534
training based on swo takes 925.4301273822784 seconds


##Yelp

In [0]:
swo_test(
    yelp_train_dataset,
    yelp_test_dataset,
    batch_size = 500,
    num_classes = yelp_num_classes,
    data_type = yelp_data_type,
    encoder = yelp_encoder,
    epochs = 3
)

transforming data to dataset object
swo takes 82.04675245285034 seconds
Epoch 1/3
448/448 [==============================] - 722s 2s/step - loss: 0.4160 - accuracy: 0.7668 - val_loss: 0.2180 - val_accuracy: 0.9202
Epoch 2/3
448/448 [==============================] - 718s 2s/step - loss: 0.1896 - accuracy: 0.9323 - val_loss: 0.1972 - val_accuracy: 0.9257
Epoch 3/3
142/448 [========>.....................] - ETA: 7:41 - loss: 0.1678 - accuracy: 0.9411

# Poisson

In [0]:
def generate_batches_by_poisson(dataset, batch_size, data_type = 'image', test_mode = False):
    dataset_size = int(tf.data.experimental.cardinality(dataset))
    steps = dataset_size // batch_size
    ratio = batch_size / dataset_size
    
    indexes = []
    for _ in range(steps):
      pros = np.random.uniform(0,1,dataset_size)
      chosen = [i for i in range(dataset_size) if pros[i] < ratio]
      indexes.append(chosen)
    
    if test_mode:
      returned_indexes = indexes.copy()
    
    batch_group_by_index = {}
    for batch_index in range(steps):
      for element_index in indexes[batch_index]:
        # steps * batch_size = n, so O(n)
        if element_index not in batch_group_by_index:
          batch_group_by_index[element_index] = [batch_index]
        else:
          batch_group_by_index[element_index].append(batch_index)

    element_index = 0
    batch_features = [[] for _ in range(steps)]
    batch_labels = [[] for _ in range(steps)]

    if data_type == 'image':
      for element in dataset.as_numpy_iterator():
        if element_index in batch_group_by_index:
          for batch_index in batch_group_by_index[element_index]:
            # on average, O(n*1) = O(n), on worst case, O(n*n/m) = O(n^2)
            # the worst case is not gonna happen, statistically
            batch_features[batch_index].append(element[0])
            batch_labels[batch_index].append(element[1])
        element_index += 1
    
      features_shape = [None, 64, 64, 3]
    
    if data_type == 'text':
      max_text_len = 0
      for element in dataset.as_numpy_iterator():
        if element_index in batch_group_by_index:
          for batch_index in batch_group_by_index[element_index]:
            # on average, O(n*1) = O(n), on worst case, O(n*n/m) = O(n^2)
            # the worst case is not gonna happen, statistically
            batch_features[batch_index].append(element[0])
            batch_labels[batch_index].append(element[1])
            max_text_len = max(len(element[0]), max_text_len)
        element_index += 1
      
      #padding
      for bf in batch_features:
        for i in range(len(bf)):
          bf[i] = list(bf[i]) + [0]*(max_text_len-len(bf[i]))
      
      features_shape = [None, max_text_len]

    print("transforming data to dataset object")

    labels_shape = [None, ]

    dataset = tf.data.Dataset.from_generator(
        lambda: zip(batch_features,batch_labels),
        (tf.float32, tf.float32),
        (features_shape, labels_shape)
    )

    if not test_mode:
      return dataset
    else:
      return dataset, returned_indexes

In [0]:
def poisson_test(train_dataset, test_dataset, batch_size = 50, epochs = 10, data_type = 'image', num_classes = 2, encoder = None):
  train_dataset_size = int(tf.data.experimental.cardinality(train_dataset))
  test_dataset_size = int(tf.data.experimental.cardinality(test_dataset))
  train_steps = train_dataset_size // batch_size
  test_steps = test_dataset_size // batch_size

  if data_type == 'image':
    poisson_test_dataset = test_dataset.batch(batch_size).repeat()
    model_poisson = cnn_model(num_classes)
  
  if data_type == 'text':
    poisson_test_dataset = test_dataset.padded_batch(batch_size).repeat()
    model_poisson = rnn_model(encoder, num_classes)

  t1_poisson = time.time()
  poisson_train_dataset = generate_batches_by_poisson(train_dataset, batch_size=batch_size, data_type=data_type).repeat()
  t2_poisson = time.time()
  print("poisson takes", t2_poisson-t1_poisson, "seconds")

  t3_poisson = time.time()
  model_poisson.fit(
    poisson_train_dataset,
    steps_per_epoch=train_steps,
    validation_data = poisson_test_dataset,
    validation_steps = test_steps,
    epochs=epochs
  )
  t4_poisson = time.time()
  print("training based on poisson takes", t4_poisson-t3_poisson, "seconds")

##Cat_versus_dog

In [0]:
poisson_test(catdog_train_dataset, catdog_test_dataset, num_classes = catdog_num_classes, data_type = catdog_data_type)

transforming data to dataset object
poisson takes 15.386031866073608 seconds
Epoch 1/10
372/372 [==============================] - 14s 38ms/step - loss: 0.6561 - accuracy: 0.6538 - val_loss: 0.6058 - val_accuracy: 0.7015
Epoch 2/10
372/372 [==============================] - 9s 25ms/step - loss: 0.5350 - accuracy: 0.7525 - val_loss: 0.5318 - val_accuracy: 0.7551
Epoch 3/10
372/372 [==============================] - 9s 25ms/step - loss: 0.4916 - accuracy: 0.7853 - val_loss: 0.5062 - val_accuracy: 0.7809
Epoch 4/10
372/372 [==============================] - 9s 25ms/step - loss: 0.4592 - accuracy: 0.8111 - val_loss: 0.4836 - val_accuracy: 0.7987
Epoch 5/10
372/372 [==============================] - 9s 25ms/step - loss: 0.4400 - accuracy: 0.8258 - val_loss: 0.4765 - val_accuracy: 0.8077
Epoch 6/10
372/372 [==============================] - 9s 25ms/step - loss: 0.4168 - accuracy: 0.8421 - val_loss: 0.4609 - val_accuracy: 0.8232
Epoch 7/10
372/372 [==============================] - 9s 25ms/st

##Flowers

In [0]:
poisson_test(flower_train_dataset, flower_test_dataset, num_classes = flower_num_classes, data_type = flower_data_type)

transforming data to dataset object
poisson takes 2.613868474960327 seconds
Epoch 1/10
58/58 [==============================] - 2s 33ms/step - loss: 1.5197 - accuracy: 0.3410 - val_loss: 1.3262 - val_accuracy: 0.4514
Epoch 2/10
58/58 [==============================] - 2s 29ms/step - loss: 1.2200 - accuracy: 0.5144 - val_loss: 1.2127 - val_accuracy: 0.5029
Epoch 3/10
58/58 [==============================] - 2s 28ms/step - loss: 1.0997 - accuracy: 0.5807 - val_loss: 1.1203 - val_accuracy: 0.5429
Epoch 4/10
58/58 [==============================] - 2s 29ms/step - loss: 1.0025 - accuracy: 0.6192 - val_loss: 1.0385 - val_accuracy: 0.5943
Epoch 5/10
58/58 [==============================] - 2s 29ms/step - loss: 0.9119 - accuracy: 0.6628 - val_loss: 0.9914 - val_accuracy: 0.6171
Epoch 6/10
58/58 [==============================] - 2s 28ms/step - loss: 0.8493 - accuracy: 0.6848 - val_loss: 1.0083 - val_accuracy: 0.6071
Epoch 7/10
58/58 [==============================] - 2s 29ms/step - loss: 0.789

##Malaria

In [0]:
poisson_test(malaria_train_dataset, malaria_test_dataset, num_classes = malaria_num_classes, data_type = malaria_data_type)

transforming data to dataset object
poisson takes 16.633229732513428 seconds
Epoch 1/10
440/440 [==============================] - 11s 24ms/step - loss: 0.4922 - accuracy: 0.7704 - val_loss: 0.2553 - val_accuracy: 0.9262
Epoch 2/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1971 - accuracy: 0.9457 - val_loss: 0.2187 - val_accuracy: 0.9440
Epoch 3/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1643 - accuracy: 0.9538 - val_loss: 0.1873 - val_accuracy: 0.9516
Epoch 4/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1525 - accuracy: 0.9562 - val_loss: 0.1842 - val_accuracy: 0.9505
Epoch 5/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1427 - accuracy: 0.9590 - val_loss: 0.1841 - val_accuracy: 0.9500
Epoch 6/10
440/440 [==============================] - 10s 22ms/step - loss: 0.1355 - accuracy: 0.9598 - val_loss: 0.1761 - val_accuracy: 0.9540
Epoch 7/10
440/440 [==============================] - 10s 2

##Imdb

In [0]:
poisson_test(
    imdb_train_dataset,
    imdb_test_dataset,
    batch_size = 1000,
    num_classes = imdb_num_classes,
    data_type = imdb_data_type,
    encoder = imdb_encoder,
    epochs = 5
)

transforming data to dataset object
poisson takes 7.836404085159302 seconds
Epoch 1/5
100/100 [==============================] - 127s 1s/step - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6927 - val_accuracy: 0.4938
Epoch 2/5
100/100 [==============================] - 126s 1s/step - loss: 0.6830 - accuracy: 0.5130 - val_loss: 0.6350 - val_accuracy: 0.7146
Epoch 3/5
100/100 [==============================] - 125s 1s/step - loss: 0.5088 - accuracy: 0.7551 - val_loss: 0.4541 - val_accuracy: 0.7938
Epoch 4/5
100/100 [==============================] - 124s 1s/step - loss: 0.3274 - accuracy: 0.8787 - val_loss: 0.4193 - val_accuracy: 0.8228
Epoch 5/5
100/100 [==============================] - 124s 1s/step - loss: 0.2434 - accuracy: 0.9182 - val_loss: 0.4545 - val_accuracy: 0.8242
training based on poisson takes 639.0689625740051 seconds


##Yelp

In [0]:
poisson_test(
    yelp_train_dataset,
    yelp_test_dataset,
    batch_size = 500,
    num_classes = yelp_num_classes,
    data_type = yelp_data_type,
    encoder = yelp_encoder,
    epochs = 3
)

transforming data to dataset object
poisson takes 94.86305785179138 seconds
Epoch 1/3
448/448 [==============================] - 601s 1s/step - loss: 0.4375 - accuracy: 0.7546 - val_loss: 0.2275 - val_accuracy: 0.9154
Epoch 2/3
448/448 [==============================] - 602s 1s/step - loss: 0.1913 - accuracy: 0.9325 - val_loss: 0.2028 - val_accuracy: 0.9253
Epoch 3/3
448/448 [==============================] - 605s 1s/step - loss: 0.1575 - accuracy: 0.9462 - val_loss: 0.2036 - val_accuracy: 0.9239
training based on poisson takes 1817.2364497184753 seconds


# **Test**
test whether swo and poisson sampling methods perform normal:
*   whether swo and poisson generate enough batches, 1000 in this case
*   whether swo and poisson generate batches of correct shapes
  *   swo has shape of (50, 32, 32, 3)
  *   poisson has shape of (viriable size around, 32, 32, 3)
*   whether swo and poisson can appropriately sample examples from the dataset







##swo test

In [0]:
swo_train_dataset_2, swo_indexes = generate_batches_by_swo(catdog_train_dataset, 50, test_mode=True)
swo_train_dataset_2

transforming data to dataset object


<FlatMapDataset shapes: ((50, 64, 64, 3), (50,)), types: (tf.float32, tf.float32)>

In [0]:
batch_num = 0
for _ in swo_train_dataset_2.as_numpy_iterator():
  batch_num += 1
print(batch_num)

372


In [0]:
for s in swo_train_dataset_2.take(3).as_numpy_iterator():
  print("batch features shape:", np.shape(s[0]), "batch labels shape:", np.shape(s[1]))

batch features shape: (50, 64, 64, 3) batch labels shape: (50,)
batch features shape: (50, 64, 64, 3) batch labels shape: (50,)
batch features shape: (50, 64, 64, 3) batch labels shape: (50,)


In [0]:
for ind in swo_indexes[:3]:
  print(ind)
  print()

[13796  1790  8050  8811 16246  6228  7534  7708 13424  2269  6260  5984
  1543  4552  9910  1433 13618  6401  2161 15890  2616 17588  4106  7839
  6198  7871  8469  1193  1002  9803 18171 18196  7030 18010  7505  6201
 10630 14166 15310 16373 15203  7890 16796 15359 11673 16090  6770 16268
 16364 12855]

[11893 14620 17294 13886  7415 14286  7145   528 18392  2040  9569 16079
 17323  4061 15995  8305 11762 15201  9292  2202 16028 17930  1898 15857
 11122  9540  5683 18235   462 14076 13722   545  2071  4055 10426  3179
  8283  5362 17200 17902   339 14898 12611 11927  3161 10116 10169 10713
 13377  7958]

[16845 18496 10580 11019 11136 13526 11309  3603 15019 13786  5729  7208
  6767 11638   130  2290  6519 14560  4056  4462   538  4814  2834 17256
 14947 14592   230 13234 11048  8851  4208  7173  7531 18201 11679 16642
 13879  1784 16099  3540  4842  6914  9967 14082  6693 13219 12171  9924
  6533  8133]



so, each batch has 50 examples

##poisson test

In [0]:
poisson_train_dataset_2, poisson_indexes = generate_batches_by_poisson(catdog_train_dataset, 50, test_mode=True)
poisson_train_dataset_2

transforming data to dataset object


<FlatMapDataset shapes: ((None, 64, 64, 3), (None,)), types: (tf.float32, tf.float32)>

In [0]:
batch_num = 0
for p in poisson_train_dataset_2.as_numpy_iterator():
  batch_num += 1
print(batch_num)

372


In [0]:
for p in poisson_train_dataset_2.take(5).as_numpy_iterator():
  print("batch features shape:", np.shape(p[0]), "batch labels shape:", np.shape(p[1]))

batch features shape: (48, 64, 64, 3) batch labels shape: (48,)
batch features shape: (46, 64, 64, 3) batch labels shape: (46,)
batch features shape: (48, 64, 64, 3) batch labels shape: (48,)
batch features shape: (45, 64, 64, 3) batch labels shape: (45,)
batch features shape: (42, 64, 64, 3) batch labels shape: (42,)


In [0]:
for ind in poisson_indexes[:3]:
  print(np.array(ind))
  print()

[  446   680   914  1088  1246  1436  2131  2332  2443  2904  3006  3833
  3835  4037  4331  4602  4844  5788  5872  6599  6679  7860  8253  8371
  8932  9800 10446 10489 10506 10669 10852 11196 12167 13852 14147 14446
 15203 16060 16626 16734 16854 17286 17399 17423 18301 18365 18483 18536]

[  346  1310  1448  3356  3595  3734  3764  4480  5276  6237  6421  6636
  6909  7008  7213  7590  7613  7888  8560  8592  8596  8776  9044  9180
  9204  9285  9724  9806  9887 10564 11547 12338 12406 12410 12687 13489
 13936 14152 14648 15499 15684 15818 15828 17391 17901 18349]

[  347   982  1661  1709  1862  1949  2517  2929  2967  3008  4451  5814
  6171  6574  6963  7492  7810  7887  8507  8532  8585  9595  9706 10013
 11383 11778 11893 12025 12875 13578 13871 14408 14417 14617 14640 14755
 15102 15280 15297 15463 15473 16062 16241 16448 17252 17338 17759 18079]

